In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import pprint
import gmaps

# Import API key
from gkey import gkey 

gmaps.configure(api_key=gkey)

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lasa,46.6166,10.7002,6.49,51,49,0.68,IT,1667884296
1,1,verkhnyaya inta,65.9811,60.3094,-2.73,94,100,3.42,RU,1667884297
2,2,luderitz,-26.6481,15.1594,17.61,79,50,3.28,NaN,1667884297
3,3,castro,-24.7911,-50.0119,8.34,99,13,2.17,BR,1667884298
4,4,bluff,-46.6000,168.3333,18.78,74,50,6.97,NZ,1667884298


In [4]:
# Finding null data
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     578 non-null    int64  
 1   City        578 non-null    object 
 2   Lat         578 non-null    float64
 3   Lng         578 non-null    float64
 4   Max Temp    578 non-null    float64
 5   Humidity    578 non-null    int64  
 6   Cloudiness  578 non-null    int64  
 7   Wind Speed  578 non-null    float64
 8   Country     572 non-null    object 
 9   Date        578 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.3+ KB


In [5]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

#df['points'] = df['points'].astype(float)

# Fill NaN values and convert to float
city_data_df["Humidity"] = city_data_df['Humidity'].astype(float)

In [6]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     578 non-null    int64  
 1   City        578 non-null    object 
 2   Lat         578 non-null    float64
 3   Lng         578 non-null    float64
 4   Max Temp    578 non-null    float64
 5   Humidity    578 non-null    float64
 6   Cloudiness  578 non-null    int64  
 7   Wind Speed  578 non-null    float64
 8   Country     572 non-null    object 
 9   Date        578 non-null    int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 45.3+ KB


In [7]:
humid = city_data_df['Humidity']
humid

0      51.0
1      94.0
2      79.0
3      99.0
4      74.0
       ... 
573    40.0
574    89.0
575    89.0
576    90.0
577    46.0
Name: Humidity, Length: 578, dtype: float64

In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,dissipating=False, max_intensity=max(city_data_df["Humidity"]),point_radius = 2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"]>=21)&(city_data_df["Max Temp"]<=27)&(city_data_df["Cloudiness"]<=0)&(city_data_df["Wind Speed"]<=4.5),:]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,cabo san lucas,22.8909,-109.9124,23.06,73.0,0,0.89,MX,1667884343
248,248,makakilo city,21.3469,-158.0858,25.18,70.0,0,2.57,US,1667884426
294,294,najran,17.4924,44.1277,21.99,25.0,0,1.74,SA,1667884366
448,448,praia,14.9215,-23.5087,22.30,83.0,0,2.57,CV,1667884518


In [10]:
#df2 = df[['Courses', 'Fee']].copy()

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City","Country","Lat","Lng","Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
88,cabo san lucas,MX,22.8909,-109.9124,73.0,
248,makakilo city,US,21.3469,-158.0858,70.0,
294,najran,SA,17.4924,44.1277,25.0,
448,praia,CV,14.9215,-23.5087,83.0,


In [11]:
# Set params:
params = {
    "radius": 10000,
    "types": "hotel",
    "keyword": "hotels",
    "key": gkey
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name
88,cabo san lucas,MX,22.8909,-109.9124,73.0,"The Cape, A Thompson Hotel"
248,makakilo city,US,21.3469,-158.0858,70.0,Hampton Inn & Suites Oahu/Kapolei
294,najran,SA,17.4924,44.1277,25.0,The District Hotel فندق ذا دستركت
448,praia,CV,14.9215,-23.5087,83.0,Hotel Oasis Atlantico Praiamar


In [12]:
info_box = """



Name: {Hotel Name}
City: {City}
Country: {Country}



"""



hotel_information = [info_box.format(**row) for index, row in hotel_df.iterrows()]
hotel_location = hotel_df[["Lat", "Lng"]]

In [13]:
hotel_plotting = gmaps.marker_layer(hotel_location,info_box_content= hotel_information)



fig = gmaps.figure()
fig.add_layer(hotel_plotting)



fig

Figure(layout=FigureLayout(height='420px'))